In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os

# 데이터 가져오기

In [6]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'Untitled.ipynb',
 '서울시 역사마스터 정보.csv',
 '카드 데이터 EDA.ipynb']

In [16]:
subway_info = pd.read_csv('서울시 역사마스터 정보.csv', encoding='cp949')
subway_info.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488


# 주소 빼오기

## 방법1 : 상호명으로 주소 찾기

In [2]:
searching = "합정 스타벅스"
size = 1
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}&size={}'.format(searching, size)
headers = {"Authorization": "KakaoAK ce19ca6be45d63e2ec27de006ee373e9"}
places = requests.get(url, headers = headers).json()['documents']
places

[{'address_name': '서울 마포구 서교동 395-166',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '26572121',
  'phone': '1522-3232',
  'place_name': '스타벅스 서교점',
  'place_url': 'http://place.map.kakao.com/26572121',
  'road_address_name': '서울 마포구 양화로 78',
  'x': '126.916980454434',
  'y': '37.5514601750423'}]

In [26]:
def convert_adr(searching):
    size = 1
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}&size={}'.format(searching, size)
    headers = {"Authorization": "KakaoAK ce19ca6be45d63e2ec27de006ee373e9"}
    places = requests.get(url, headers = headers).json()['documents']
    return places

In [13]:
def convert_ad(searching):
    size = 1
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}&size={}'.format(searching, size)
    headers = {"Authorization": "KakaoAK ce19ca6be45d63e2ec27de006ee373e9"}
    places = requests.get(url, headers = headers).json()['documents']
    return " ".join(places[0]['road_address_name'].split()[0:2])

In [15]:
subway_info.역사명.head().apply(convert_ad)

0          
1          
2    서울 강서구
3    경기 김포시
4    경기 김포시
Name: 역사명, dtype: object

==> 역명이지만, 말미에 '역'이란 이름이 안붙어서 잘 못찾음 전처리 필요함

In [20]:
convert_name = subway_info.역사명.apply(lambda x: str(x)+"역")
convert_name.head()

0      미사역
1      강일역
2    김포공항역
3      고촌역
4      풍무역
Name: 역사명, dtype: object

In [30]:
# 너무 양이 양이 많으니 5개만 빼서 바꿔보자!

convert_name.head().apply(convert_ad)

0    경기 하남시
1    서울 강동구
2    서울 강서구
3    경기 김포시
4    경기 김포시
Name: 역사명, dtype: object

In [35]:
convert_name.apply(convert_ad)

IndexError: list index out of range

==> 중간에 주소를 못 찾는 건지 위도, 경도로 주소를 뽑는 방법으로 주소를 빼오자!

## 위도/경도로 주소 찾기

In [36]:
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK ce19ca6be45d63e2ec27de006ee373e9"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address_name']
    return str(match_first)

In [72]:
subway_info

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...,...
733,154,종로5가,1호선,127.001849,37.570926
734,153,종로3가,1호선,126.991847,37.570406
735,152,종각,1호선,126.982923,37.570161
736,151,시청,1호선,126.977088,37.565715


In [41]:
lat_lon_to_addr(127.193877,37.560927)

'경기도 하남시 망월동'

In [73]:
subway_info.위도

0      127.193877
1      127.175930
2      126.801868
3      126.770345
4      126.732387
          ...    
733    127.001849
734    126.991847
735    126.982923
736    126.977088
737    126.972135
Name: 위도, Length: 738, dtype: float64

In [76]:
subway_info.경도

0      37.560927
1      37.557490
2      37.562360
3      37.601243
4      37.612488
         ...    
733    37.570926
734    37.570406
735    37.570161
736    37.565715
737    37.556228
Name: 경도, Length: 738, dtype: float64

In [78]:
주소 = []
for i in range(738):
    result = lat_lon_to_addr(subway_info.위도[i], subway_info.경도[i])
    주소.append(result)

In [81]:
pd.Series(data=주소)

0          경기도 하남시 망월동
1        서울특별시 강동구 강일동
2        서울특별시 강서구 방화동
3      경기도 김포시 고촌읍 신곡리
4          경기도 김포시 사우동
            ...       
733     서울특별시 종로구 종로5가
734     서울특별시 종로구 종로3가
735     서울특별시 종로구 종로1가
736        서울특별시 중구 정동
737     서울특별시 중구 봉래동2가
Length: 738, dtype: object

In [84]:
subway_info["주소"] = pd.Series(data=주소)

In [85]:
subway_info.head(2)

,역사_ID,역사명,호선,위도,경도,주소
0,9996,미사,5호선,127.193877,37.560927,경기도 하남시 망월동
1,9995,강일,5호선,127.175930,37.557490,서울특별시 강동구 강일동


# 내보내기

In [86]:
subway_info.to_csv("지하철역_정보.csv")

In [87]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'Untitled.ipynb',
 '서울시 역사마스터 정보.csv',
 '지하철역_정보.csv',
 '카드 데이터 EDA.ipynb']